In [ ]:
import pandas as pd
import numpy as np
import re

The original dataset

In [ ]:
fix_path = r""
df = pd.read_csv(fix_path)

In [ ]:
df = df.dropna(subset=['Department', 'Symptom', 'Category'])

CleanText

As the sentences distribution are messy in some way. The main here idea is to first, remove all dots, truncate all trailing spaces. Then by the 
logic that all dot must followed by an UpperCased letter, except for the first letter in the sentence 


In [ ]:
# Test function before applying
text = "A.A.A. ."
re.sub(pattern=r"\.", repl=" ", string=text)

In [ ]:
def cleanText(text):
    text = text.replace('\xa0', ' ')
    text = re.sub(pattern=r"\.", repl=" ", string=text) # Remove dots
    text = re.sub(pattern=r"([A-Z])", repl=r". \1", string=text) # Adding dot before uppercased letters
    text = re.sub(pattern=r"[~!@#$%^&*()<>?,/'\";:\{\[\]\}+=\_\-\*\+]", repl="", string=text) # Remove puncs
    text = re.sub(pattern=r"(  +)", repl=" ", string=text) # Remove trailing spaces
    return text

In [47]:
df['Symptom'] = df['Symptom'].apply(lambda x:cleanText(x))

In [49]:
# Remove the leading dot
prefix_dot = df[df['Symptom'].str[0] == '.'].index

for idx in prefix_dot:
    df.loc[idx, 'Symptom'] = df.loc[idx, 'Symptom'].replace(".", "", 1).lstrip()

In [ ]:
[x for x in df['Symptom']]

Concat duplicate sentences

+ Removing duplicated sentences using set. Consider lowercasing all letters for clearer data :>

In [50]:
def concatSymptom(text):
   sentences = set([text.strip().lower() for text in text.split(".")])
   return ' '.join([sentence for sentence in sentences if sentence is not None])

In [51]:
df['Symptom'] = df['Symptom'].apply(lambda x: concatSymptom(x))

Unify Label

In [ ]:
df['Department'].value_counts()

In [ ]:
df['Category'].value_counts()

In [ ]:

def LabelUnifier(label):
    departments = {
    "Thần kinh - Tâm thần": [
        "Thần kinh", "Nội thần kinh", "Ngoại thần kinh", "Tâm thần", "Tâm lý"
    ],
    "Hô hấp": [
        "Hô hấp"
    ],
    "Truyền nhiễm": [
        "Truyền nhiễm"
    ],
    "Tiết niệu - Nam khoa": [
        "Tiết niệu", "Thận tiết niệu", "Nam khoa", "Nam học"
    ],
    "Nội tiết": [
        "Nội tiết", "Nội tiết chuyển hóa"
    ],
    "Tiêu hóa - Gan mật": [
        "Tiêu hóa", "Tiêu hóa - Gan mật"
    ],
    "Tim mạch - Mạch máu": [
        "Tim mạch", "Mạch máu"
    ],
    "Ung thư": [
        "Ung thư", "Ung bướu"
    ],
    "Di truyền": [
        "Di truyền"
    ],
    "Răng hàm mặt": [
        "Răng hàm mặt"
    ],
    "Tai mũi họng": [
        "Tai mũi họng"
    ],
    "Da liễu": [
        "Da liễu", "Da tóc móng"
    ],
    "Nhãn khoa": [
        "Mắt", "Nhãn khoa"
    ],
    "Cơ xương khớp": [
        "Cơ xương khớp"
    ],
    "Dị ứng": [
        "Dị ứng"
    ],
    "Xét nghiệm": [
        "Xét nghiệm"
    ],
    "Sản phụ khoa": [
        "Sản phụ khoa", "Sức khỏe sinh sản", "Hỗ trợ sinh sản ivf", "Mang thai"
    ],
    "Máu": [
        "Máu", "Máu miễn dịch"
    ],
    "Ngoại lồng ngực": [
        "Ngoại lồng ngực"
    ]
}
    for department, keywords in departments.items():
        if label in keywords: return department

    return 0


In [ ]:
df['Department'] = df['Department'].apply(lambda x:LabelUnifier(x))

In [ ]:
df = df[~(df['Department'] == 0)]

In [ ]:
df = df[['Symptom', 'Department', 'Category']]

In [53]:
df.to_csv(fix_path, index=False)